<a href="https://colab.research.google.com/github/660710078/TopGun2025/blob/main/Problem1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ติดตั้ง libraries ที่จำเป็น
!pip install ultralytics
!pip install opencv-python-headless
!pip install numpy pandas matplotlib

# Import libraries
import cv2
import numpy as np
from ultralytics import YOLO
import matplotlib.pyplot as plt
from google.colab import drive
from google.colab.patches import cv2_imshow
import math



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.9 MB/s eta 0:00:00
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# กำหนด path
base_path = '/content/drive/MyDrive/CANET/TGR_DRONE_DATASET'
yaml_path = f'{base_path}/data.yaml'

# ตรวจสอบโครงสร้าง dataset
!ls -la {base_path}

total 14
-rw------- 1 root root  272 Nov 11 08:27 data.yaml
-rw------- 1 root root  151 Nov 11 08:27 README.dataset.txt
-rw------- 1 root root  895 Nov 11 08:27 README.roboflow.txt
drwx------ 2 root root 4096 Nov 11 08:30 test
drwx------ 2 root root 4096 Nov 11 08:30 train
drwx------ 2 root root 4096 Nov 11 08:30 valid


In [ ]:
# # โหลด YOLOv8 model
model = YOLO('yolov8n.pt')  # n=nano, s=small, m=medium, l=large, x=xlarge

# Train model
results = model.train(
    data=yaml_path,
    epochs=150,          # รอบ train, ปรับตาม dataset
    imgsz=640,           # ขนาดภาพ, 640 หรือ 768 เพื่อ detect small object
    batch=16,            # batch size, ปรับตาม VRAM
    name='drone_detection',
    patience=20,         # early stopping
    save=True,
    plots=True,
    augment=True,        # ใช้ data augmentation
    lr0=0.001,           # learning rate เริ่มต้น
    flipud=0.5,          # flip vertical
    fliplr=0.5,          # flip horizontal
    hsv_h=0.015,         # hue augmentation
    hsv_s=0.7,           # saturation augmentation
    hsv_v=0.4,           # value augmentation
    translate=0.1,       # shift ภาพ
    scale=0.5,           # scale ภาพ
    degrees=5.0          # rotate ภาพเล็กน้อย
)

Ultralytics 8.3.227 🚀 Python-3.12.12 torch-2.8.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/CANET/TGR_DRONE_DATASET/data.yaml, degrees=5.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.5, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=drone_detection3, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patienc

In [ ]:
best_model_path = (f'{base_path}/best_drone_model.pt')
model = YOLO(best_model_path)

In [ ]:
# ประเมินผลบน validation set
val_metrics = model.val(data=yaml_path, split='val')

# แสดงผล metric
print(f"   mAP50:     {val_metrics.box.map50:.4f}")
print(f"   mAP50-95:  {val_metrics.box.map:.4f}")
print(f"   Precision: {val_metrics.box.mp:.4f}")
print(f"   Recall:    {val_metrics.box.mr:.4f}")

In [ ]:
# import cv2
# from ultralytics import YOLO

# โฟลเดอร์ภาพ test
image_folder = 'test_images'
image_list = [f for f in os.listdir(image_folder) if f.endswith(('.jpg', '.png'))]

# ชื่อไฟล์ CSV ที่จะบันทึก
csv_file = 'drone_predictions.csv'

# เตรียมเขียน CSV
with open(csv_file, mode='w', newline='') as f:
    writer = csv.writer(f)

    # loop ทุกภาพ
    for image_name in image_list:
        image_path = os.path.join(image_folder, image_name)
        results = model.predict(source=image_path, imgsz=640, conf=0.25, iou=0.45)

        # loop result ของแต่ละภาพ
        for result in results:
            boxes = result.boxes.xyxy.cpu().numpy()  # [x1, y1, x2, y2]
            for box in boxes:
                x1, y1, x2, y2 = box
                width = x2 - x1
                height = y2 - y1
                cx = int(x1)
                cy = int(y1)
                w = int(width)
                h = int(height)

                row = [image_name, cx, cy, w, h]

                # เขียนลง CSV
                writer.writerow(row)

                # print ตาม format
                print("----------")
                print(row)